In [ ]:
import numpy as np
import scipy.ndimage as ndi
import holoviews as hv
import matplotlib.pyplot as plt
#import nd2reader
import skimage
import numba
import timeit

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

In [ ]:
from paulssonlab.shenker.image_analysis import blur

For discussions of algorithms, see:
- https://stackoverflow.com/questions/98359/fastest-gaussian-blur-implementation
- https://github.com/bfraboni/FastGaussianBlur
- http://blog.ivank.net/fastest-gaussian-blur.html
- https://dsp.stackexchange.com/questions/50576/fastest-available-algorithm-to-blur-an-image-low-pass-filter

# Setup

In [ ]:
nd2 = nd2reader.ND2Reader("/home/jqs1/scratch/jqs1/microscopy/220704/220704rbs_library_fish.nd2")
img = nd2.get_frame_2D(v=50, t=20, c=2)

In [ ]:
img = skimage.io.imread("blur/rfp.tiff")

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img)

# Benchmarks

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(blur.scipy_box(img, 10))

In [ ]:
img.shape

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(blur.scipy_box_transpose(img, 10))

In [ ]:
timeit.Timer("blur.scipy_box_blur_2d(img, 10)", globals=globals()).timeit(5)

In [ ]:
timeit.Timer("blur.scipy_box_blur(img, 10)", globals=globals()).timeit(5)

In [ ]:
timeit.Timer("blur.scipy_box_blur(img, 10)", globals=globals()).timeit(5)

# Performance

# Accuracy matrix

# Test

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img2)

In [ ]:
img2 = img[:500,:500].copy()
blur._numba_stack_blur1d(img2, 20)
#img2 = blur.scipy_box_blur(img2, 10, mode="nearest")
#img2 = ndi.filters.gaussian_filter1d(img2, 10, mode="nearest")
img2

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img2)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img2)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img2)

In [ ]:
%%timeit
img3 = ndi.gaussian_filter(img, 10)

In [ ]:
img2 = img.copy()
ref = blur.numba_stack_blur(img2, 10)

In [ ]:
%%timeit
img2 = img.copy()

In [ ]:
%%timeit
img2 = img.copy()
out = blur.numba_stack_blur(img2, 10)

In [ ]:
blur._numba_stack_blur1d.signatures

In [ ]:
blur._numba_stack_blur1d.inspect_types(pretty=True)

In [ ]:
print(list(blur._numba_stack_blur1d.inspect_asm().items())[0][1])

In [ ]:
find_instr(blur._numba_stack_blur1d, keyword='subp', sig=0)

In [ ]:
img2 = img.copy()
out = blur.numba_stack_blur(img2, 10)

In [ ]:
np.allclose(ref, out)

In [ ]:
def find_instr(func, keyword, sig=0, limit=5):
    count = 0
    for l in func.inspect_asm(func.signatures[sig]).split('\n'):
        if keyword in l:
            count += 1
            print(l)
            if count >= limit:
                break
    if count == 0:
        print('No instructions found')

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(img2)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(img3)

# Prototyping

In [ ]:
@numba.njit(fastmath=True, error_model="numpy")
def sb(ary, radius):
    dtype = ary.dtype
    width = ary.shape[-1]
    width_minus1 = width - 1
    height = ary.shape[-2]
    radius_plus1 = radius + 1
    div = 2 * radius + 1
    div_sum = (radius + 1) * (radius + 1)
    sum_factor = (radius + 1) * (radius + 2) // 2
    new_row = np.empty(width, dtype=dtype)
    stack = np.empty(div, dtype=dtype)
    for y in range(height):
        sum_ = 0
        sum_in = 0
        sum_out = 0
        value = ary[y, 0]
        sum_out = radius_plus1 * value
        sum_ += sum_factor * value
        for i in range(radius_plus1):
            stack[i] = value
        for i in range(1, radius_plus1):
            value = ary[y, width_minus1 if i > width_minus1 else i]
            stack[i + radius] = value
            sum_ += value * (radius_plus1 - i)
            sum_in += value
        stack_idx = radius
        for x in range(width):
            # TODO: can also use lookup tables for multiplication/division
            new_row[x] = np.round(sum_ / div_sum)
            sum_ -= sum_out
            stack_start = (stack_idx + div - radius) % div
            stack_value = stack[stack_start]
            sum_out -= stack_value
            xp = (x + radius_plus1) % width_minus1
            value = ary[y, xp]
            stack[stack_start] = value
            sum_in += value
            sum_ += sum_in
            stack_idx = (stack_idx + 1) % div
            stack_value = stack[stack_idx]
            sum_out += stack_value
            sum_in -= stack_value
        # copy new_row to row
        for x in range(width):
            ary[y, x] = new_row[x]

sb(img.copy(), 10)

### New

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

### Ref

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
%%timeit
sb(img2, 10)

In [ ]:
sb.inspect_types(pretty=True)

In [ ]:
print(list(sb.inspect_asm().items())[0][1])

In [ ]:
@numba.njit(fastmath=True, error_model="numpy")
def frac_diff1(x, y):
    out = np.empty_like(x)
    for i in range(x.shape[0]):
        out[i] = 2 * (x[i] - y[i]) / (x[i] + y[i])
    return out

In [ ]:
x32 = np.linspace(1, 2, 10000, dtype=np.float32)
y32 = np.linspace(2, 3, 10000, dtype=np.float32)

In [ ]:
frac_diff1(x32, y32)

In [ ]:
%%timeit
frac_diff1(x32, y32)

In [ ]:
print(list(frac_diff1.inspect_asm().items())[0][1])